In [6]:
import pandas as pd
import numpy as np
import copy
import collections as clc
import math
from numpy.random import permutation

cols = ['methodology', 'requirements_volatility', 
        'requirements_clarity', 'dev_time', 'project_size', 'team_size', 
        'prod_complexity', 'testing_intensity', 'risk_analysis', 'user_participation',
        'team_expertise', 'dev_expertise', 'doc_needed', 'fund_avail', 'delivery_speed', 'task_visualisation']
        
num_cols = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

df = pd.read_csv('SDLC2.csv', names = cols, usecols=num_cols, header = 0)

df.tail()


,methodology,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed,task_visualisation
121,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
122,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
123,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
124,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
125,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low


In [7]:
from numpy.random import permutation
import math

random_indices = permutation(df.index)
test_cutoff = math.floor(len(df)/5)
print(test_cutoff)
test = df.loc[random_indices[1:test_cutoff]]
test1 =  df.tail(8)
train = df.loc[random_indices[test_cutoff:]]
test1

25


,methodology,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed,task_visualisation
118,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
119,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
120,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
121,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
122,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
123,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
124,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low
125,Waterfall,Fixed,understandable/early defined,Non-Intensive,Small,Small (1-5),Simple,After development is done (Non-intensive testing),High,Low,Medium,Medium,High,Low,Low,Low


In [8]:
min_significance = 0.7
max_star_size = 5


def fit_CN2_Model(train):
    """
    Function to fit the CN2 model to 
    """

    #import ipdb;ipdb.set_trace(context=8)
            
    selectors = rule_creation()
    remaining_examples = train
    rule_list = []
    # loop until data is all covered.
    while len(remaining_examples) >= 1:
        best_new_calculate_significance = 1
        rules_to_specialise = []
        considered_results = pd.DataFrame()
        #search rule space until rule best_new_calculate_significance = 1significance is lower than user set boundary(0.5 for testing)
        while  best_new_calculate_significance > 0.5:
            #calls statement if its first iteration of loop
            if len(rules_to_specialise) == 0:
                ordered_rule_results = apply_heuristics(selectors, remaining_examples)
                trimmed_rule_results = ordered_rule_results[0:max_star_size]
            elif len(rules_to_specialise) != 0: 		 
                specialised_rules = specialisation_process(rules_to_specialise,selectors)
                #import ipdb;ipdb.set_trace(context=8)
                ordered_rule_results = apply_heuristics(specialised_rules, remaining_examples)
                trimmed_rule_results = ordered_rule_results[0:max_star_size]
            #append newly discovered rules to existing ones, order them and then take best X(3 for testing)
            considered_results = considered_results.append(trimmed_rule_results)
            considered_results =  order_rule_list(considered_results).iloc[0:2]
            #update 'rules to specialise' and significance value of best new rule 
            rules_to_specialise = trimmed_rule_results['rule']
            best_new_calculate_significance = trimmed_rule_results['significance'].values[0]
            #ipdb.set_trace(context=8)
        
        #import ipdb;ipdb.set_trace(context=8)
        best_rule = (considered_results['rule'].iloc[0],considered_results['predict_class'].iloc[0],
                        considered_results['num_insts_covered'].iloc[0])
        best_rule_coverage_index, best_rule_coverage_df = complex_coverage(best_rule[0], remaining_examples)
        rule_list.append(best_rule)
        remaining_examples = remaining_examples.drop(best_rule_coverage_index)

    return rule_list 

def test_model(rule_list, data_set = 'default'):
    """
    Test rule list returned by fit_CN2_Model function on test data(or manually supplied data)
    returns a dataframe that contains the rule, rule acc, num of examples covered.
    Also return general accuracy as average of each rule accuracy
    """
    if type(data_set) == str:
        data_set = test

    remaining_examples = data_set
    list_of_row_dicts = []

    for rule in rule_list:
        rule_coverage_indexes,rule_coverage_dataframe = complex_coverage(rule[0],remaining_examples)
        #check for zero coverage due to noise(lense data too small)
        if len(rule_coverage_dataframe) == 0:
            row_dictionary = {'rule':rule,'pred_class':'zero coverage','rule_acc':0,
                            'num_examples':0,'num_correct':0,
                            'num_wrong':0}
            list_of_row_dicts.append(row_dictionary)			   
        #otherwise generate statistics about rule then save and remove examples from the data and test next rule.
        else:				   
            class_of_covered_examples = rule_coverage_dataframe['methodology']
            #import ipdb;ipdb.set_trace(context=8)
            class_counts = class_of_covered_examples.value_counts()
            rule_accuracy = class_counts.values[0]/sum(class_counts)
            num_correctly_classified_examples = class_counts.values[0]
            num_incorrectly_classified_examples = sum(class_counts.values) - num_correctly_classified_examples

            row_dictionary = {'rule':rule,'pred_class':rule[1],'rule_acc':rule_accuracy,
                                'num_examples':len(rule_coverage_indexes),'num_correct':num_correctly_classified_examples,
                                'num_wrong':num_incorrectly_classified_examples}
            list_of_row_dicts.append(row_dictionary) 

            remaining_examples = remaining_examples.drop(rule_coverage_indexes)

    results = pd.DataFrame(list_of_row_dicts)
    overall_accuracy = sum(results['rule_acc'])/len([r for r in results['rule_acc'] if r !=0])
    return results, overall_accuracy


def apply_heuristics( complexes, data_set = 'default'):
    """
    A function which takes a list of complexes/rules and returns a pandas DataFrame
    that contains the complex, the entropy, the significance, the number of selectors,
    the number of examples covered, the length of the rule and the predicted class of the rule. 
    The input param complexes should be a list of lists of tuples.
    """
    #import ipdb;ipdb.set_trace(context=8)
    
    #build a dictionary for each rule with relevant stats
    if type(data_set) == str: 
        data_set = train
    list_of_row_dicts = []
    for row in complexes:
        rule_coverage = complex_coverage(row, data_set)[1]
        rule_length = len(row)
        #test if rule covers 0 examples
        if len(rule_coverage) == 0:

            row_dictionary = {'rule': row, 'predict_class':'dud rule',
                            'entropy': 10, 'laplace_accuracy': 0,
                            'significance':0,'length':rule_length, 
                            'num_insts_covered':0,'specificity':0}
            list_of_row_dicts.append(row_dictionary)
        #calculate stats for non 0 coverage rules
        else:

            num_examples_covered = len(rule_coverage)
            entropy_of_rule = calculate_entropy(rule_coverage)
            significance_of_rule = calculate_significance(rule_coverage)
            laplace_accuracy_of_rule = calculate_laplace_accuracy(rule_coverage)
            class_attrib = rule_coverage['methodology']
            #import ipdb;ipdb.set_trace(context=8)
            class_counts = class_attrib.value_counts()
            majority_class = class_counts.axes[0][0]
            rule_specificity = class_counts.values[0]/sum(class_counts) 
            row_dictionary = {'rule': row, 'predict_class':majority_class,
                                'entropy': entropy_of_rule, 'laplace_accuracy': laplace_accuracy_of_rule,
                                'significance':significance_of_rule,'length':rule_length, 
                                'num_insts_covered':num_examples_covered,'specificity':rule_specificity}
            list_of_row_dicts.append(row_dictionary)
    #put dictionaries into dataframe and order them according to laplace acc, length
    rules_and_stats = pd.DataFrame(list_of_row_dicts)
    ordered_rules_and_stats = order_rule_list(rules_and_stats)	

    return ordered_rules_and_stats


def order_rule_list(dataFrame_of_rules):
    """
    Function to order a dataframe of rules and stats according to laplace acc and length then reindex
    the ordered frame.
    """
    ordered_rules_and_stats = dataFrame_of_rules.sort_values(['entropy', 'length',
                                                                'num_insts_covered'], ascending=[True, True, False])
    ordered_rules_and_stats = ordered_rules_and_stats.reset_index(drop=True)	

    return ordered_rules_and_stats	


def rule_creation():
    """function to return the first set 
        of complexes which are the
        1 attribute selectors
    """

    #get attribute names
    attributes = train.columns.values.tolist()
        
    #remove class from list of attributes
    del attributes[0]
    
    #get possible values for attributes 
    possAttribVals = {} 	
    for att in attributes:
        possAttribVals[att] = set(train[att])

    # get list of attribute,value pairs
    # from possAttribVals dictionary
    attrib_value_pairs = []
    for key in possAttribVals.keys():
        for possVal in possAttribVals[key]:
            attrib_value_pairs.append([(key,possVal)])

    return attrib_value_pairs

def specialisation_process(target_complexes, selectors):
    """
    Function to specialise the complexes in the "star", the current set of
    complexes in consideration. Expects to receive a complex (a list of tuples)
    to which it adds addtional conjunctions using all the possible selectors. Returns
    a list of new, specialised complexes.
    """
    #import ipdb;ipdb.set_trace(context=8)

    provisional_specialisations = []
    for targ_complex in target_complexes: 
        for selector in selectors:
            #check to see if target complex is a single tuple otherwise assume list of tuples
            if type(targ_complex) == tuple:
                comp_to_specialise = [copy.copy(targ_complex)]
            else:
                comp_to_specialise = copy.copy(targ_complex)	
            
            comp_to_specialise.append(selector[0])
            
            #count if any selector is duplicated and append rule if not
            count_of_selectors_in_complex = clc.Counter(comp_to_specialise)
            flag = True
            for count in count_of_selectors_in_complex.values():
                if count > 1:
                    flag = False


            if flag == True:
                provisional_specialisations.append(comp_to_specialise)
    

    #remove complexes that have been specialised with same selector eg [(A=1),(A=1)]
    #trimmed_specialisations = [rule for rule in provisional_specialisations if rule[0] != rule[1]]

    return provisional_specialisations		


def attribute_value_pair(passed_complex):
    """
    build a rule in dict format where target attributes have a single value and non-target attributes
    have a list of all possible values. Checks if there are repetitions in the attributes used, if so
    it returns False
    """
    atts_used_in_rule = []
    for selector in passed_complex:
        atts_used_in_rule.append(selector[0])
    set_of_atts_used_in_rule = set(atts_used_in_rule)
        
    if len(set_of_atts_used_in_rule) < len(atts_used_in_rule):
        return False


    rule = {}
    attributes = train.columns.values.tolist() 	
    for att in attributes:
        rule[att] = list(set(train[att]))

    for att_val_pair in passed_complex:
        att = att_val_pair[0]
        val = att_val_pair[1]
        rule[att] = [val]
    return rule		


def complex_coverage(passed_complex, data_set = 'default'):
    """ Returns set of instances of the data 
        which complex(rule) covers as a dataframe.
    """
    if type(data_set) == str: 
        data_set = train
    coverage = []

    rule = attribute_value_pair(passed_complex)
    if rule == False:
        return [],[]

    mask = data_set.isin(rule).all(axis=1)
    rule_coverage_indexes = data_set[mask].index.values
    rule_coverage_dataframe = data_set[mask]
    
    # #iterate over dataframe rows
    # for index,row in data_set.iterrows():
    # 	if check_rule_datapoint(row, complex):
    # 		coverage.append(index)
    

    #import ipdb;ipdb.set_trace(context=8)
    return rule_coverage_indexes, rule_coverage_dataframe

def check_rule_datapoint( datapoint, complex):
    """
    Function to check if a given data point satisfies
    the conditions of a given complex. Data point 
    should be a pandas Series. Complex should be a
    tuple or a list of tuples where each tuple is of
    the form ('Attribute', 'Value').
    """
    if type(complex) == tuple:
        if datapoint[complex[0]] == complex[1]:
            return True
        else:
            return False
    #import ipdb;ipdb.set_trace(context=8)
    if type(complex) == list:
        result = True	
        for selector in complex:
            if datapoint[selector[0]] != selector[1]:
                result = False

        return result

def calculate_entropy(covered_data):
    """
    Function to check the Shannon entropy of a complex/rule
    given the instances it covers. Pass the instances 
    covered by the rule as a dataframe where class cloumn is
    named class.
    """ 
    class_series = covered_data['methodology']
    num_instances = len(class_series)
    class_counts = class_series.value_counts()
    class_probabilities = class_counts.divide(num_instances)
    log2_of_classprobs = np.log2(class_probabilities)
    plog2p = class_probabilities.multiply(log2_of_classprobs)
    entropy = plog2p.sum()*-1
    
    return entropy

def calculate_significance(covered_data):
    """
    Fucntion to check the significance of a rule using the 
    likelihood ratio test where observed frequency of class
    in the coverage of the rule is compared to the observed
    frequencies of the classes in the training data.
    """
    covered_classes = covered_data['methodology']
    covered_num_instances = len(covered_classes)
    covered_counts = covered_classes.value_counts()
    covered_probs = covered_counts.divide(covered_num_instances)

    train_classes = train['methodology']
    train_num_instances = len(train_classes)
    train_counts = train_classes.value_counts()
    train_probs = train_counts.divide(train_num_instances)

    significance = covered_probs.multiply(np.log(covered_probs.divide(train_probs))).sum()*2

    return significance

def calculate_laplace_accuracy(covered_data):
    """
    function to calculate laplace accuracy of a rule, taken from update to CN2
    paper by author of original CN2.
    """
    #import ipdb;ipdb.set_trace(context=8)

    class_series = covered_data['methodology']
    class_counts = class_series.value_counts()
    num_instances = len(class_series)
    num_classes = len(class_counts)
    num_pred_class = class_counts.iloc[0]
    #laplace_accuracy = (num_pred_class+1)/(num_instances+num_classes)
    laplace_accuracy_2 = (num_instances + num_classes - num_pred_class - 1)/(num_instances + num_classes)
    return laplace_accuracy_2


In [9]:
lenseFit = fit_CN2_Model(df)

KeyboardInterrupt: 

In [ ]:
lenseFit

[([('project_size', 'Small'),
   ('project_type',
    'System (sits between the hardware and the application software e.g. OSs)')],
  'Waterfall',
  28),
 ([('project_size', 'Small')], 'Scrum', 2),
 ([('testing_intensity', 'After development is done (Non-intensive testing)'),
   ('project_type', 'Application (everything else)')],
  'RAD',
  22),
 ([('doc_needed', 'Low'), ('project_size', 'Medium')], 'Scrum', 20),
 ([('doc_needed', 'Low'),
   ('project_type',
    'Utility (performs specific tasks to keep the computer running e.g. antivirus)')],
  'Spiral',
  6),
 ([('doc_needed', 'Low')], 'Hybrid: Scrum and Kanban', 2),
 ([('team_size', 'Small (1-5)'), ('project_size', 'Medium')], 'Scrum', 6),
 ([('testing_intensity', 'After development is done (Non-intensive testing)')],
  'Hybrid: Scrum and Kanban',
  3),
 ([('team_size', 'Small (1-5)')], 'Hybrid: Scrum and Waterfall', 2),
 ([('requirements_volatility', 'Fixed'),
   ('project_type', 'Application (everything else)')],
  'RAD',
  2),
 (

In [ ]:
testSet = [['Spiral', 'Utility (performs specific tasks to keep the computer running e.g. antivirus)', 'Changing', 'unknown/defined later in the lifecycle', 
            'Intensive', 'Large', 'Small (1-5)', 'Complex', 'After each cycle (Intensive testing)', 'High', 'Low','High', 'High', 
            'Low','Low','Medium']]

testo = pd.DataFrame(testSet, columns=cols)
testo

,methodology,project_type,requirements_volatility,requirements_clarity,dev_time,project_size,team_size,prod_complexity,testing_intensity,risk_analysis,user_participation,team_expertise,dev_expertise,doc_needed,fund_avail,delivery_speed
0,Spiral,Utility (performs specific tasks to keep the c...,Changing,unknown/defined later in the lifecycle,Intensive,Large,Small (1-5),Complex,After each cycle (Intensive testing),High,Low,High,High,Low,Low,Medium


In [ ]:
lenseTest = test_model(lenseFit,testo)[0]
print(lenseTest)

                                                 rule     pred_class  \
0   ([(project_size, Small), (project_type, System...  zero coverage   
1                 ([(project_size, Small)], Scrum, 2)  zero coverage   
2   ([(testing_intensity, After development is don...  zero coverage   
3   ([(doc_needed, Low), (project_size, Medium)], ...  zero coverage   
4   ([(doc_needed, Low), (project_type, Utility (p...         Spiral   
5   ([(doc_needed, Low)], Hybrid: Scrum and Kanban...  zero coverage   
6   ([(team_size, Small (1-5)), (project_size, Med...  zero coverage   
7   ([(testing_intensity, After development is don...  zero coverage   
8   ([(team_size, Small (1-5))], Hybrid: Scrum and...  zero coverage   
9   ([(requirements_volatility, Fixed), (project_t...  zero coverage   
10    ([(requirements_volatility, Fixed)], Kanban, 1)  zero coverage   
11                 ([(risk_analysis, Low)], Scrum, 1)  zero coverage   
12  ([(team_expertise, Medium), (team_size, Large ...  zero cove